# 1. Description

Data description sourced from the Kaggle competition page.

### train.csv
- `county` - An ID code for the county.
- `is_business` - Boolean for whether or not the prosumer is a business.
- `product_type` - ID code with the following mapping of codes to contract types: `{0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}`.
- `target` - The consumption or production amount for the relevant segment for the hour. The segments are defined by the `county`, `is_business`, and `product_type`.
- `is_consumption` - Boolean for whether or not this row's target is consumption or production.
- `datetime` - The Estonian time in EET (UTC+2) / EEST (UTC+3).
- `data_block_id` - All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictins made on October 31st is 100 then the historic weather `data_block_id` for October 31st will be 101 as the historic weather data is only actually available the next day.
- `row_id` - A unique identifier for the row.
- `prediction_unit_id` - A unique identifier for the `county`, `is_business`, and `product_type` combination. *New prediction units can appear or disappear in the test set*.

### gas_prices.csv

- `origin_date` - The date when the day-ahead prices became available.
- `forecast_date` - The date when the forecast prices should be relevant.
- `[lowest/highest]_price_per_mwh` - The lowest/highest price of natural gas that on the day ahead market that trading day, in Euros per megawatt hour equivalent.
- `data_block_id`

### client.csv
- `product_type`
- `county` - An ID code for the county. See `county_id_to_name_map.json` for the mapping of ID codes to county names.
- `eic_count` - The aggregated number of consumption points (EICs - European Identifier Code).
- `installed_capacity` - Installed photovoltaic solar panel capacity in kilowatts.
- `is_business` - Boolean for whether or not the prosumer is a business.
- `date`
- `data_block_id`

### electricity_prices.csv
- `origin_date`
- `forecast_date` - Represents the start of the 1-hour period when the price is valid
- `euros_per_mwh` - The price of electricity on the day ahead markets in euros per megawatt hour.
- `data_block_id`

### forecast_weather.csv
Weather forecasts that would have been available at prediction time. Sourced from the <u>[European Centre for Medium-Range Weather Forecasts](https://codes.ecmwf.int/grib/param-db/?filter=grib2)</u>.

- `[latitude/longitude]` - The coordinates of the weather forecast.
- `origin_datetime` - The timestamp of when the forecast was generated.
- `hours_ahead` - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
- `temperature` - The air temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
- `dewpoint` - The dew point temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
- `cloudcover_[low/mid/high/total]` - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total. Estimated for the end of the 1-hour period.
- `10_metre_[u/v]_wind_component` - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second. Estimated for the end of the 1-hour period.
- `data_block_id`
- `forecast_datetime` - The timestamp of the predicted weather. Generated from `origin_datetime` plus `hours_ahead`. This represents the start of the 1-hour period for which weather data are forecasted.
- `direct_solar_radiation` - The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the hour, in watt-hours per square meter.
- `surface_solar_radiation_downwards` - The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, accumulated during the hour, in watt-hours per square meter.
- `snowfall` - Snowfall over hour in units of meters of water equivalent.
- `total_precipitation` - The accumulated liquid, comprising rain and snow that falls on Earth's surface over the described hour, in units of meters.

### historical_weather.csv
<u>[Historic weather data](https://open-meteo.com/en/docs)</u>.

- `datetime` - This represents the start of the 1-hour period for which weather data are measured.
- `temperature` - Measured at the end of the 1-hour period.
- `dewpoint` - Measured at the end of the 1-hour period.
- `rain` - Different from the forecast conventions. The rain from large scale weather systems of the hour in millimeters.
- `snowfall` - Different from the forecast conventions. Snowfall over the hour in centimeters.
- `surface_pressure` - The air pressure at surface in hectopascals.
- `cloudcover_[low/mid/high/total]` - Different from the forecast conventions. Cloud cover at 0-3 km, 3-8, 8+, and total.
- `windspeed_10m` - Different from the forecast conventions. The wind speed at 10 meters above ground in meters per second.
- `winddirection_10m` - Different from the forecast conventions. The wind direction at 10 meters above ground in degrees.
- `shortwave_radiation` - Different from the forecast conventions. The global horizontal irradiation in watt-hours per square meter.
- `direct_solar_radiation`
- `diffuse_radiation` - Different from the forecast conventions. The diffuse solar irradiation in watt-hours per square meter.
- `[latitude/longitude]` - The coordinates of the weather station.
- `data_block_id`

# 2. Import

In [ ]:
import colorcet as cc
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from matplotlib import dates as mdates
from pandas.core.groupby.generic import DataFrameGroupBy
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
RAND = 10
DATA_PATH = "../raw_data/"

train_df = pd.read_csv(f"{DATA_PATH}train.csv")
gas_prices_df = pd.read_csv(f"{DATA_PATH}gas_prices.csv")
client_df = pd.read_csv(f"{DATA_PATH}client.csv")
electricity_prices_df = pd.read_csv(f"{DATA_PATH}electricity_prices.csv")
forecast_weather_df = pd.read_csv(f"{DATA_PATH}forecast_weather.csv")
historical_weather_df = pd.read_csv(f"{DATA_PATH}historical_weather.csv")
station_county_mapping = pd.read_csv(
    f"{DATA_PATH}weather_station_to_county_mapping.csv"
)
county_id_to_name_map = pd.read_json(
    f"{DATA_PATH}county_id_to_name_map.json",
    typ="series",
).str.capitalize()

In [ ]:
DF_DICT = {
    "train_df": train_df,
    "gas_prices_df": gas_prices_df,
    "client_df": client_df,
    "electricity_prices_df": electricity_prices_df,
    "forecast_weather_df": forecast_weather_df,
    "historical_weather_df": historical_weather_df,
}
CATEGORICAL_DICT = {
    "county": county_id_to_name_map,
    "is_business": {0: "not_business", 1: "business"},
    "is_consumption": {0: "production", 1: "consumption"},
    "product_type": {
        0: "combined",
        1: "fixed",
        2: "general_service",
        3: "spot",
    },
}
COLORS_LIST = (
    cc.glasbey[:4]
    + [cc.glasbey[8]]
    + cc.glasbey[5:8]
    + [cc.glasbey[4]]
    + [cc.glasbey[12]]
    + cc.glasbey[10:12]
    + [cc.glasbey[9]]
    + cc.glasbey[13:16]
)
PALETTE = sns.color_palette(COLORS_LIST)

In [ ]:
pd.set_option(
    "display.float_format",
    lambda x: f"{x:.2e}" if abs(x) < 0.01 and x != 0 else f"{x:.2f}",
)

# 3. Data Preparation

From description:
- `datetime` - The Estonian time in EET (UTC+2) / EEST (UTC+3).
- `data_block_id` - All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictins made on October 31st is 100 then the historic weather `data_block_id` for October 31st will be 101 as the historic weather data is only actually available the next day.
- `row_id` - A unique identifier for the row.
- `prediction_unit_id` - A unique identifier for the `county`, `is_business`, and `product_type` combination. *New prediction units can appear or disappear in the test set*.

Also competition host <u>[provided](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455833)</u> this scheme:
***
Let’s say we are on day D at 11am. We want to predict next day D+1 net consumption from 00 to 23 for every hours.
<table style="border: 1px solid black; border-collapse: collapse; width: 100%;">
  <tr>
    <th style="border: 1px solid black;">Category</th>
    <th style="border: 1px solid black;">Weather forecast</th>
    <th style="border: 1px solid black;">Historical weather</th>
    <th style="border: 1px solid black;">Historical consumption and production / Client data</th>
    <th style="border: 1px solid black;">Electricity prices</th>
    <th style="border: 1px solid black;">Gas prices</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">Last available data</td>
    <td style="border: 1px solid black;">Forecast for every hours of D and D+1 (published on D)</td>
    <td style="border: 1px solid black;">Every hours until day D, 10 am</td>
    <td style="border: 1px solid black;">Every hours of Day D-1</td>
    <td style="border: 1px solid black;">Every hours of day D (published on D-1)</td>
    <td style="border: 1px solid black;">Data for day D (published on D-1)</td>
  </tr>
</table>

Prices are published everyday at 2 pm (so after 11 am), that is we do not have D+1 prices.
The data_block_id already reflects this timeline of availability of the data. There is no need to apply additional lag if joining on data_block_id.
***

Later, I will explore whether there are any discrepancies in the correlations between `data_block_id` and `datetime`, or between `prediction_unit_id` and the `categorical features`. For now, I will analyze the data by:
1. Checking all DataFrames using `describe()` and `info()`.
2. Verifying the presence of missing values or duplicates.

I will use the following functions for initial analysis and processing.

In [ ]:
def print_centered_header(text: str) -> None:
    """
    Prints a line of 100 characters with the given text centered in
    it and filled with '<>'.
    """
    total_length = 100
    text = f" {text.strip()} "
    side_length = (total_length - len(text)) // 2
    print(
        "<" * side_length
        + text
        + ">" * (total_length - len(text) - side_length)
    )

In [ ]:
def df_show_info(df: pd.DataFrame) -> None:
    """
    Display DataFrame summary, NaNs and duplicated values.
    """

    print_centered_header("HEAD")
    display(df.head())

    print_centered_header("INFO")
    df.info(show_counts=True)

    print_centered_header("DESCRIBE")
    display(df.describe(include="all"))

    print_centered_header("MISSING VALUES")
    nan_counts = df.isna().sum()
    nan_series = nan_counts[nan_counts > 0]
    if nan_series.empty:
        print("No missing values")
    else:
        display(nan_series)

    print_centered_header("DUPLICATES")
    dup_counts = df.duplicated().sum()
    if dup_counts:
        display(dup_counts)
    else:
        print("No duplicate rows")

In [ ]:
def categorical_mapper(
    df: pd.DataFrame,
    values_mapper: dict,
) -> pd.DataFrame:
    """
    Map categorical columns in the DataFrame to their corresponding
    values based on the provided mapper and convert these columns to
    categorical type.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame to process.
    values_mapper : dict
        A dictionary where keys are column names, and values are
        mapping dictionaries.

    Returns
    -------
    pd.DataFrame
        Modified DataFrame with specified columns mapped to their
        categorical values.
    """
    for key, value in values_mapper.items():
        if key in df.columns:
            df[key] = df[key].map(value).astype("category")
        else:
            print(f"Column '{key}' not found in DataFrame, skipping.")

    return df

In [ ]:
def mismatched_indices_check(
    date_diff: pd.Series,
    id_diff: pd.Series,
    difference: np.float64,
) -> None:
    """
    Check if rows with an n-day difference in the date_diff have
    mismatched indices with id_diff rows that have difference of n.
    """

    print(
        f"Difference {difference}: ",[
            "no mismatches",
            "indices mismatch detected"][
        int(np.any(
            date_diff[date_diff == pd.Timedelta(difference, "day")]
            .index
            != id_diff[id_diff == difference].index))],
        ".",
        sep="",
    )

In [ ]:
def same_groups(dfgb: DataFrameGroupBy, column: str) -> pd.Series:
    """
    Combine values of the specified column in each subgroup into tuples,
    compare these tuples within each group, and return the count of
    duplicated and non-duplicated tuples.
    """
    return dfgb[column].agg(tuple).duplicated().value_counts()

## train_df

In [ ]:
df_show_info(train_df)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< HEAD >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.71,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.59,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.00,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.31,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.90,0,2021-09-01 00:00:00,0,4,2


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INFO >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   county              2018352 non-null  int64  
 1   is_business         2018352 non-null  int64  
 2   product_type        2018352 non-null  int64  
 3   target              2017824 non-null  float64
 4   is_consumption      2018352 non-null  int64  
 5   datetime            2018352 non-null  object 
 6   data_block_id       2018352 non-null  int64  
 7   row_id              2018352 non-null  int64  
 8   prediction_unit_id  2018352 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DESCRIBE >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
count,2018352.00,2018352.00,2018352.00,2017824.00,2018352.00,2018352,2018352.00,2018352.00,2018352.00
unique,NaN,NaN,NaN,NaN,NaN,15312,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,2022-11-27 12:00:00,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,138,NaN,NaN,NaN
mean,7.30,0.54,1.90,274.86,0.50,NaN,321.87,1009175.50,33.05
std,4.78,0.50,1.08,909.50,0.50,NaN,182.63,582648.18,19.59
min,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00
25%,3.00,0.00,1.00,0.38,0.00,NaN,166.00,504587.75,16.00
50%,7.00,1.00,2.00,31.13,0.50,NaN,323.00,1009175.50,33.00
75%,11.00,1.00,3.00,180.21,1.00,NaN,479.00,1513763.25,50.00


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MISSING VALUES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


target    528
dtype: int64

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DUPLICATES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No duplicate rows


- There is a small number of missing values (528/2,018,352 < 0.1%) and no duplicates in the DataFrame. Only the `target` feature contains missing values, which can likely be handled using interpolation or mean imputation to ensure data consistency.
- The categorical features `county`, `is_business`, `product_type`, and `is_consumption` can be converted to the categorical data type. Additionally, their values can be renamed to improve clarity and understanding.
- The `target` feature can be converted to `float32` for memory optimization.
- The `datetime` feature can be converted to `datetime64[ns]`.
- The `data_block_id` feature can be converted to `uint16` (range: 0 through 65,535) due to its small maximum value.
- The `row_id` feature appears to be similar to the index values in the current default sorting. This feature could potentially be deleted in the future if it is not used for data merging later. For now, it can be converted to `uint32` (range: 0 through 4,294,967,295) as its range fits well within this data type. The use of `int64` is unnecessary because it supports negative numbers, which are irrelevant in this case.
- The `prediction_unit_id` feature can be converted to `uint8` (range: 0 through 255). Although new combinations may appear in the future, this data type is sufficient to uniquely represent all future combinations of current county, is_business, and product_type (16 * 2 * 4 = 96).

### Data Transformation

In [ ]:
# Rename to avoid confusion and improve readability
train_df = categorical_mapper(train_df, CATEGORICAL_DICT)

# Change data types to reduce memory usage
train_df = train_df.astype(
    {
        "target": "float32",
        "data_block_id": "uint16",
        "row_id": "uint32",
        "prediction_unit_id": "uint8",
        "datetime": "datetime64[ns]",
    }
)

# Rename to distinct from datetime-like features from other DFs
train_df = train_df.rename(columns={"datetime": "target_datetime"})

### Missing Values

In [ ]:
train_df[train_df.isna().any(axis=1)].head()

,county,is_business,product_type,target,is_consumption,target_datetime,data_block_id,row_id,prediction_unit_id
178938,Harjumaa,not_business,fixed,NaN,production,2021-10-31 03:00:00,60,178938,0
178939,Harjumaa,not_business,fixed,NaN,consumption,2021-10-31 03:00:00,60,178939,0
178940,Harjumaa,not_business,general_service,NaN,production,2021-10-31 03:00:00,60,178940,1
178941,Harjumaa,not_business,general_service,NaN,consumption,2021-10-31 03:00:00,60,178941,1
178942,Harjumaa,not_business,spot,NaN,production,2021-10-31 03:00:00,60,178942,2


The `datetime` values for rows with missing target values start from '2021-10-31 03:00:00', rather than from '2021-09-01 00:00:00'.

In [ ]:
na_datetimes = train_df[train_df.isna().any(axis=1)].target_datetime.unique()
na_datetimes

<DatetimeArray>
['2021-10-31 03:00:00', '2022-03-27 03:00:00', '2022-10-30 03:00:00',
 '2023-03-26 03:00:00']
Length: 4, dtype: datetime64[ns]

All missing values correspond to the start or end of daylight saving time.

In [ ]:
train_df.isna().values.sum() == (
    train_df.loc[
        train_df["target_datetime"].isin(na_datetimes), ["target"]
    ].shape[0]
)

True

All 528 target values at these timestamps are missing.

### Consistency and Relationship Validation
#### Row Identifier

In [ ]:
(train_df["row_id"] != train_df.index).sum()

0

All `row_id` values are equal to index values.

#### Datetime and data_block_id

For predictions made for day D + 1, all historical consumption and production data should have `data_block_id` values equal to D - 1. This is because the `data_block_id` represents the data available at a specific time. Data for day D is unavailable at the time of prediction, as it corresponds to the current day, and no historical data exists for it yet.

Therefore, `target_datetime` and `data_block_id` columns require a check to ensure that, in chronological order, both the date and id values show uniform growth. That is, if one date corresponds to multiple `data_block_id` values, it will be noticeable, as the differences between rows in these two columns will not be uniform.

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in target_datetime and data_block_id
train_date_diff = train_df["target_datetime"].dt.date.diff()
train_id_diff = train_df["data_block_id"].diff()

# Print unique values to verify that default order reflects sorting
# from oldest to newest without skips
print("Date differences:", train_date_diff.unique())
print("Data block ID differences:", train_id_diff.unique())

Date differences: <TimedeltaArray>
[NaT, '0 days', '1 days']
Length: 3, dtype: timedelta64[ns]
Data block ID differences: [nan  0.  1.]


- `NaT` and `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.
- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.

In [ ]:
for difference in train_id_diff.unique():
    mismatched_indices_check(train_date_diff, train_id_diff, difference)

Difference nan: no mismatches.
Difference 0.0: no mismatches.
Difference 1.0: no mismatches.


The default order of the raw data for this DataFrame is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `target_datetime` column.

#### Categorical Features and Their Identifiers

In [ ]:
# Check that each combination of county, product_type, and
# is_business corresponds to exactly one unique prediction_unit_id
(
    train_df.groupby(
        [
            "county",
            "product_type",
            "is_business",
        ],
        observed=True,
    )["prediction_unit_id"].nunique()
    != 1
).sum()

0

All combinations of `county`, `is_business`, and `product_type` correspond to a single `prediction_unit_id`, with no errors.

## gas_prices_df

In [ ]:
df_show_info(gas_prices_df)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< HEAD >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INFO >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   forecast_date          637 non-null    object 
 1   lowest_price_per_mwh   637 non-null    float64
 2   highest_price_per_mwh  637 non-null    float64
 3   origin_date            637 non-null    object 
 4   data_block_id          637 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 25.0+ KB
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DESCRIBE >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
count,637,637.00,637.00,637,637.00
unique,637,NaN,NaN,637,NaN
top,2021-09-01,NaN,NaN,2021-08-31,NaN
freq,1,NaN,NaN,1,NaN
mean,NaN,95.04,107.75,NaN,319.00
std,NaN,47.55,54.74,NaN,184.03
min,NaN,28.10,34.00,NaN,1.00
25%,NaN,60.00,67.53,NaN,160.00
50%,NaN,85.21,93.47,NaN,319.00
75%,NaN,109.00,130.74,NaN,478.00


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MISSING VALUES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No missing values
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DUPLICATES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No duplicate rows


- There are no missing values or duplicates in the DataFrame.
- The `data_block_id` values start from 1, unlike the `data_block_id` values in train_df, which start from 0. This difference exists because </u>[`[lowest/highest]_price_per_mwh` are end-of-day prices and aren't available in the late morning when forecasts are made](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/453355#2515054)</u>.

### Data Transformation

In [ ]:
# Change data types to reduce memory usage
gas_prices_df = gas_prices_df.astype(
    {
        "forecast_date": "datetime64[ns]",
        "lowest_price_per_mwh": "float32",
        "highest_price_per_mwh": "float32",
        "origin_date": "datetime64[ns]",
        "data_block_id": "uint16",
    }
)

# Rename to distinct from datetime-like features from other DFs
gas_prices_df = gas_prices_df.rename(
    columns={
        "forecast_date": "gas_forecast_date",
        "origin_date": "gas_origin_date",
    }
)

### Consistency and Relationship Validation
#### Origin and forecast dates
Due to the strong correlation between the two datetime columns (at least based on the description and a few observed rows), it's necessary to verify if this correlation holds true.

In [ ]:
# Check that each origin date is the next day compared to the previous
# one

gas_date_diff = gas_prices_df["gas_origin_date"].diff()
print(gas_date_diff.unique())

<TimedeltaArray>
[NaT, '1 days']
Length: 2, dtype: timedelta64[ns]


- Yes, all values are chronological days without skips. The `NaT` value corresponds to the first date, where no previous date exists for comparison.

In [ ]:
# Check if any forecast dates are not exactly one day after origin dates
(
    gas_prices_df["gas_origin_date"]
    + pd.Timedelta(days=1)
    != gas_prices_df["gas_forecast_date"]
).sum()

0

- All `gas_origin_date` values are correct and represent the day before their corresponding `gas_forecast_date` values. This means that correlation between these features is 1, and one of them can likely be deleted.
- The raw data in this DataFrame is sorted chronologically by datetime.

#### Origin date and data_block_id

In [ ]:
gas_id_diff = gas_prices_df["data_block_id"].diff()
print(gas_id_diff.unique())

[nan  1.]


In [ ]:
for difference in gas_id_diff.unique():
    mismatched_indices_check(gas_date_diff, gas_id_diff, difference)

Difference nan: no mismatches.
Difference 1.0: no mismatches.


- Similar to `gas_origin_date`, the first value is missing, and all subsequent values have a difference of 1. Since the differences are consistent across both columns, it can be concluded that there are no errors between `gas_origin_date` and `data_block_id`.

## client_df

In [ ]:
df_show_info(client_df)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< HEAD >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INFO >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41919 entries, 0 to 41918
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_type        41919 non-null  int64  
 1   county              41919 non-null  int64  
 2   eic_count           41919 non-null  int64  
 3   installed_capacity  41919 non-null  float64
 4   is_business         41919 non-null  int64  
 5   date                41919 non-null  object 
 6   data_block_id       41919 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 2.2+ MB
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DESCRIBE >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
count,41919.00,41919.00,41919.00,41919.00,41919.00,41919,41919.00
unique,NaN,NaN,NaN,NaN,NaN,636,NaN
top,NaN,NaN,NaN,NaN,NaN,2022-11-26,NaN
freq,NaN,NaN,NaN,NaN,NaN,69,NaN
mean,1.90,7.30,73.35,1450.77,0.54,NaN,322.90
std,1.08,4.78,144.06,2422.23,0.50,NaN,182.08
min,0.00,0.00,5.00,5.50,0.00,NaN,2.00
25%,1.00,3.00,13.00,321.90,0.00,NaN,167.00
50%,2.00,7.00,32.00,645.20,1.00,NaN,324.00
75%,3.00,11.00,70.00,1567.15,1.00,NaN,480.00


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MISSING VALUES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No missing values
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DUPLICATES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No duplicate rows


- There are no missing values or duplicates in the DataFrame.
- The `data_block_id` values start from 2 instead of 0 or 1, as mentioned earlier.

### Data Transformation

In [ ]:
# Rename to avoid confusion and improve readability
client_df = categorical_mapper(client_df, CATEGORICAL_DICT)

# Change data types to reduce memory usage
client_df = client_df.astype(
    {
        "eic_count": "uint32",
        "installed_capacity": "float32",
        "date": "datetime64[ns]",
        "data_block_id": "uint16",
    }
)

# Rename to distinct from datetime-like features from other DFs
client_df = client_df.rename(columns={"date": "client_date"})

Column 'is_consumption' not found in DataFrame, skipping.


### Consistency and Relationship Validation
#### Date and data_block_id
As with the previous DataFrames, it is necessary to validate the correlation between `client_date` and `data_block_id`.

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in client_date and data_block_id
client_date_diff = client_df["client_date"].dt.date.diff()
client_id_diff = client_df["data_block_id"].diff()


# Print unique values to verify that default order reflects sorting
# from oldest to newest without skips
print("Date differences:", client_date_diff.unique())
print("Data block ID differences:", client_id_diff.unique())

Date differences: <TimedeltaArray>
[NaT, '0 days', '1 days']
Length: 3, dtype: timedelta64[ns]
Data block ID differences: [nan  0.  1.]


- `NaT` and `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.
- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.

In [ ]:
for difference in client_id_diff.unique():
    mismatched_indices_check(client_date_diff, client_id_diff, difference)

Difference nan: no mismatches.
Difference 0.0: no mismatches.
Difference 1.0: no mismatches.


- The default order of the raw data for this DataFrame is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `client_date` column.

## electricity_prices_df

In [ ]:
df_show_info(electricity_prices_df)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< HEAD >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INFO >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15286 entries, 0 to 15285
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   forecast_date  15286 non-null  object 
 1   euros_per_mwh  15286 non-null  float64
 2   origin_date    15286 non-null  object 
 3   data_block_id  15286 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 477.8+ KB
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DESCRIBE >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,forecast_date,euros_per_mwh,origin_date,data_block_id
count,15286,15286.00,15286,15286.00
unique,15286,NaN,15286,NaN
top,2021-09-01 00:00:00,NaN,2021-08-31 00:00:00,NaN
freq,1,NaN,1,NaN
mean,NaN,157.06,NaN,318.99
std,NaN,121.15,NaN,183.89
min,NaN,-10.06,NaN,1.00
25%,NaN,85.29,NaN,160.00
50%,NaN,128.28,NaN,319.00
75%,NaN,199.80,NaN,478.00


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MISSING VALUES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No missing values
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DUPLICATES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No duplicate rows


- There are no missing values or duplicates in the DataFrame.
- The `data_block_id` values start from 1.
- The minimum value of the `euros_per_mwh` column is negative.

### Non-positive Prices

In [ ]:
electricity_prices_df[electricity_prices_df["euros_per_mwh"] <= 0].shape

(36, 4)

According to the [<u>official comment</u>](https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/454932#2523730), such prices are not an error.

[<u>From Wikipedia</u>](https://en.wikipedia.org/wiki/Negative_pricing): negative pricing can occur when demand for a product drops or supply increases to an extent that owners or suppliers are prepared to pay others to accept it, in effect setting the price to a negative number.

### Data Transformation

In [ ]:
# Change data types to reduce memory usage
electricity_prices_df = electricity_prices_df.astype(
    {
        "forecast_date": "datetime64[ns]",
        "euros_per_mwh": "float32",
        "origin_date": "datetime64[ns]",
        "data_block_id": "uint16",
    }
)

# Rename to distinct from datetime-like features from other DFs
electricity_prices_df = electricity_prices_df.rename(
    columns={
        "forecast_date": "electricity_forecast_datetime",
        "origin_date": "electricity_origin_datetime",
    }
)

### Consistency and Relationship Validation
As with `electricity_prices_df`, it is necessary to validate the correlation between origin date, forecast date, and `data_block_id`.
#### Origin and forecast dates

In [ ]:
# Check that each electricity_origin_date is the next hour compared
# to the previous one
electricity_prices_df["electricity_origin_datetime"].diff().unique()

<TimedeltaArray>
[NaT, '0 days 01:00:00', '0 days 02:00:00']
Length: 3, dtype: timedelta64[ns]

There are also values with a 2-hour difference. This might indicate an issue with data collection or other inconsistencies.

In [ ]:
electricity_prices_df[
    electricity_prices_df["electricity_origin_datetime"].diff()
    == pd.Timedelta(2, "hour")
]

,electricity_forecast_datetime,euros_per_mwh,electricity_origin_datetime,data_block_id
4970,2022-03-27 03:00:00,100.07,2022-03-26 03:00:00,208
13705,2023-03-26 03:00:00,40.12,2023-03-25 03:00:00,572


The 2-hour time difference occurs at 03:00:00 on the dates 2022-03-27 and 2023-03-26, corresponding to the transition to daylight saving time. This transition results in missing records for the previous hour.

Nevertheless, despite these missing values, the data remains consistent on a daily basis.

In [ ]:
# Check if any forecast dates are not exactly one day after origin dates
(
    electricity_prices_df["electricity_origin_datetime"]
    + pd.Timedelta(days=1)
    != electricity_prices_df["electricity_forecast_datetime"]
).sum()

0

- All `electricity_origin_datetime` values represent the day before their corresponding `electricity_forecast_datetime` values.
- The raw data in this DataFrame is sorted chronologically by datetime.

#### Origin date and data_block_id

In [ ]:
# Create variables to store the differences between the current and
# previous row for all rows in electricity_origin_datetime and data_block_id
electricity_date_diff = electricity_prices_df[
    "electricity_origin_datetime"
].dt.date.diff()
electricity_id_diff = electricity_prices_df["data_block_id"].diff()

# Print unique values to verify that default order reflects sorting
# from oldest to newest without skips
print("Date differences:", electricity_date_diff.unique())
print("Data block ID differences:", electricity_id_diff.unique())

Date differences: <TimedeltaArray>
[NaT, '0 days', '1 days']
Length: 3, dtype: timedelta64[ns]
Data block ID differences: [nan  0.  1.]


- `NaT` and `NaN` values correspond to the first row in both columns, where no previous row data is available for comparison.
- A value of '0' indicates that the current and previous rows belong to the same day.
- A value of '1' indicates the transition from one day to the next.

In [ ]:
for difference in electricity_id_diff.unique():
    mismatched_indices_check(
        electricity_date_diff, electricity_id_diff, difference
    )

Difference nan: no mismatches.
Difference 0.0: no mismatches.
Difference 1.0: no mismatches.


- The default order of the raw data for this DataFrame is sorted by datetime, and the `data_block_id` reflects the actual data availability, with no errors observed in the `electricity_origin_datetime` column.

## forecast_weather_df

In [ ]:
df_show_info(forecast_weather_df)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< HEAD >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.60,21.70,2021-09-01 02:00:00,1,15.66,11.55,0.90,0.02,0.00,0.91,-0.41,-9.11,1,2021-09-01 03:00:00,0.00,0.00,0.00,0.00
1,57.60,22.20,2021-09-01 02:00:00,1,13.00,10.69,0.89,4.46e-03,0.00,0.89,0.21,-5.36,1,2021-09-01 03:00:00,0.00,0.00,0.00,0.00
2,57.60,22.70,2021-09-01 02:00:00,1,14.21,11.67,0.73,5.62e-03,0.00,0.73,1.45,-7.42,1,2021-09-01 03:00:00,0.00,0.00,0.00,0.00
3,57.60,23.20,2021-09-01 02:00:00,1,14.84,12.26,0.34,0.07,6.26e-04,0.39,1.09,-9.16,1,2021-09-01 03:00:00,0.00,0.00,0.00,0.00
4,57.60,23.70,2021-09-01 02:00:00,1,15.29,12.46,0.10,0.09,1.53e-05,0.18,1.27,-8.98,1,2021-09-01 03:00:00,0.00,0.00,0.00,0.00


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INFO >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424512 entries, 0 to 3424511
Data columns (total 18 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   latitude                           3424512 non-null  float64
 1   longitude                          3424512 non-null  float64
 2   origin_datetime                    3424512 non-null  object 
 3   hours_ahead                        3424512 non-null  int64  
 4   temperature                        3424512 non-null  float64
 5   dewpoint                           3424512 non-null  float64
 6   cloudcover_high                    3424512 non-null  float64
 7   cloudcover_low                     3424512 non-null  float64
 8   cloudcover_mid                     3424512 non-null  float64
 9   cloudcover_total                   3424512 non-null  fl

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
count,3424512.00,3424512.00,3424512,3424512.00,3424512.00,3424512.00,3424512.00,3424512.00,3424512.00,3424512.00,3424512.00,3424512.00,3424512.00,3424512,3424512.00,3424510.00,3424512.00,3424512.00
unique,NaN,NaN,637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15310,NaN,NaN,NaN,NaN
top,NaN,NaN,2021-09-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-30 03:00:00,NaN,NaN,NaN,NaN
freq,NaN,NaN,5376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,448,NaN,NaN,NaN,NaN
mean,58.65,24.95,NaN,24.50,5.74,2.41,0.39,0.43,0.36,0.68,1.26,0.73,319.00,NaN,151.19,110.76,2.53e-05,7.86e-05
std,0.69,2.02,NaN,13.85,7.84,7.12,0.44,0.44,0.42,0.40,4.00,4.22,183.89,NaN,256.51,187.44,1.22e-04,2.78e-04
min,57.60,21.70,NaN,1.00,-27.50,-29.68,0.00,0.00,0.00,0.00,-17.58,-22.12,1.00,NaN,-0.77,-0.33,-3.81e-06,-1.53e-05
25%,58.12,23.20,NaN,12.75,0.26,-2.36,0.00,3.36e-04,0.00,0.26,-1.47,-1.98,160.00,NaN,0.00,0.00,0.00,0.00
50%,58.65,24.95,NaN,24.50,4.87,1.84,0.09,0.23,0.10,0.98,1.47,0.94,319.00,NaN,0.00,0.60,0.00,0.00
75%,59.17,26.70,NaN,36.25,11.15,7.30,0.98,1.00,0.90,1.00,3.81,3.51,478.00,NaN,212.84,144.17,0.00,2.77e-05


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< MISSING VALUES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


surface_solar_radiation_downwards    2
dtype: int64

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< DUPLICATES >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
No duplicate rows


- There are 2 missing values and zero duplicates in the DataFrame. Only the `surface_solar_radiation_downwards` column contains missing values. 
- The `data_block_id` values start from 1.

### Data Transformation

In [ ]:
forecast_weather_df[forecast_weather_df.snowfall != 0]

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
239900,59.70,26.70,2021-10-15 02:00:00,30,6.80,3.21,0.00,0.46,0.56,0.75,11.88,4.43,45,2021-10-16 08:00:00,96.49,27.91,8.52e-05,9.33e-04
240605,58.20,22.20,2021-10-15 02:00:00,37,8.58,4.84,0.00,0.52,0.45,0.64,8.08,1.09,45,2021-10-16 15:00:00,50.61,109.79,7.38e-07,4.70e-04
240636,58.80,23.70,2021-10-15 02:00:00,37,8.45,2.29,0.00,0.43,0.78,0.79,6.33,1.83,45,2021-10-16 15:00:00,210.04,138.86,3.87e-04,3.82e-03
240717,58.20,22.20,2021-10-15 02:00:00,38,9.07,5.56,0.00,0.51,0.33,0.68,8.60,0.53,45,2021-10-16 16:00:00,240.38,113.18,-7.45e-09,5.77e-04
240765,59.10,25.20,2021-10-15 02:00:00,38,7.34,1.13,0.00,0.14,0.71,0.74,4.01,1.24,45,2021-10-16 16:00:00,207.74,94.55,9.78e-06,3.86e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294016,59.40,27.70,2023-05-06 02:00:00,35,6.58,-1.04,0.00,0.00,0.01,0.01,2.62,-3.73,613,2023-05-07 13:00:00,782.50,717.67,4.92e-07,0.00
3294017,59.40,28.20,2023-05-06 02:00:00,35,7.92,-2.04,0.00,0.09,0.27,0.28,3.03,-1.98,613,2023-05-07 13:00:00,468.47,573.81,-3.58e-07,0.00
3294026,59.70,25.70,2023-05-06 02:00:00,35,3.55,0.39,0.00,0.00,2.14e-04,2.14e-04,3.71,-1.50,613,2023-05-07 13:00:00,861.86,735.80,-4.62e-07,0.00
3294027,59.70,26.20,2023-05-06 02:00:00,35,3.73,-0.31,0.00,0.00,0.00,0.00,5.19,-2.33,613,2023-05-07 13:00:00,859.87,731.89,5.51e-07,0.00


In [ ]:
# Change data types to reduce memory usage
forecast_weather_df[["latitude", "longitude"]] = forecast_weather_df[
    ["latitude", "longitude"]
].mul(10)
forecast_weather_df = forecast_weather_df.astype(
    {
        "latitude": "uint16",
        "longitude": "uint16",
        "origin_datetime": "datetime64[ns]",
        "temperature": "float32",
        "dewpoint": "float32",
        "cloudcover_high": "float32",
        "cloudcover_low": "float32",
        "cloudcover_mid": "float32",
        "cloudcover_total": "float32",
        "10_metre_u_wind_component": "float32",
        "10_metre_v_wind_component": "float32",
        "data_block_id": "uint16",
        "forecast_datetime": "datetime64[ns]",
        "direct_solar_radiation": "float32",
        "surface_solar_radiation_downwards": "float32",
        "snowfall": "float32",
        "total_precipitation": "float32",
    }
)

forecast_weather_df["hours_ahead"] = pd.to_timedelta(
    forecast_weather_df["hours_ahead"], "h"
)

### Missing Values

In [ ]:
forecast_weather_df[forecast_weather_df.isna().any(axis=1)]

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
1849670,597,237,2022-08-11 02:00:00,0 days 03:00:00,19.04,16.85,0.91,0.00,0.00,0.91,5.91,7.62,345,2022-08-11 05:00:00,17.10,NaN,0.00,0.00
1849782,597,237,2022-08-11 02:00:00,0 days 04:00:00,18.80,16.99,0.84,0.00,0.00,0.84,5.42,8.10,345,2022-08-11 06:00:00,206.41,NaN,0.00,0.00


- Since there are only two missing values, it could be due to a local issue lasting for just 2 hours.

### Consistency and Relationship Validation
Ensure `origin_datetime`, `forecast_datetime`, `hours_ahead` values are coherent within respective `latitude` and `longitude` groups.
#### Origin datetime

In [ ]:
# Sameness check
forecast_geo_gb = forecast_weather_df.groupby(["latitude", "longitude"])
same_groups(forecast_geo_gb, "origin_datetime")

origin_datetime
True     111
False      1
Name: count, dtype: int64

This DataFrame contains 112 unique combinations of `latitude` and `longitude`, representing distinct geographical points for weather forecasts. Only one `False` value corresponds to the first group, indicating that all other location points have identical subsequences of `origin_datetime`.

In [ ]:
# Forecast creation times
forecast_weather_df["origin_datetime"].dt.time.value_counts()

origin_datetime
02:00:00    1843968
01:00:00    1580544
Name: count, dtype: int64

All forecasts are made at either 1 AM or 2 AM, likely due to shifts associated with transitions to and from daylight saving time.

In [ ]:
# Select latitude and longitude values for the first entry of the
# geographical group, as the sequences of origin_datetime values are
# identical across each group
lat0, lon0 = forecast_weather_df.iloc[0][["latitude", "longitude"]]
fw_subset_df = forecast_weather_df[
    (forecast_weather_df["latitude"] == lat0)
    & (forecast_weather_df["longitude"] == lon0)
]

# Calculate the differences between consecutive origin_datetime values
# for the selected geographical group
fod_diff = fw_subset_df["origin_datetime"].diff()

# Display the counts of unique differences between consecutive
# origin_datetime entries
fod_diff.value_counts(dropna=False)

origin_datetime
0 days 00:00:00    29939
1 days 00:00:00      632
0 days 23:00:00        2
1 days 01:00:00        2
NaT                    1
Name: count, dtype: int64

- `0 days 00:00:00`: rows corresponding to the same location and day, but with different `hours_ahead` values.
- `1 days 00:00:00`: indicates a shift from previous origin day.
- `0 days 23:00:00`: likely caused by a transition to daylight saving time.
- `1 days 01:00:00`: likely caused by a transition from daylight saving time.
- `NaT`: corresponds to the first rows in each group.

Based on these differences, it can be concluded that the data in this DataFrame is sorted by `origin_datetime` by default.

In [ ]:
# Select rows where differences in origin_datetime are not 0, 1 day, or
# NaT
unusual_diff_df = fw_subset_df[
    (fod_diff != pd.Timedelta(0))
    & (fod_diff != pd.Timedelta(1, "d"))
    & (~fod_diff.isna())
][["origin_datetime"]]

# Merge the filtered timestamps with their corresponding time
# differences
pd.merge(
    unusual_diff_df,
    fod_diff[unusual_diff_df.index].rename("difference"),
    left_index=True,
    right_index=True,
)

,origin_datetime,difference
327936,2021-11-01 01:00:00,0 days 23:00:00
1118208,2022-03-28 02:00:00,1 days 01:00:00
2284800,2022-10-31 01:00:00,0 days 23:00:00
3075072,2023-03-27 02:00:00,1 days 01:00:00


Yes, the autumn shift corresponds to a switch to winter time, and the spring shift corresponds to a switch to summer time. Specifically:
- The `origin_datetime` hour is `02:00:00` during the following periods:
  - From `2021-09-01` to `2021-10-31`
  - From `2022-03-28` to `2022-10-30`
  - From `2023-03-27` to `2023-05-30`
- The `origin_datetime` hour is `01:00:00` during the following periods:
  - From `2021-11-01` to `2022-03-27`
  - From `2022-10-31` to `2023-03-26`

#### Forecast datetime
From the data description:
- `forecast_datetime` - The timestamp of the predicted weather. Generated from `origin_datetime` plus `hours_ahead`. This represents the start of the 1-hour period for which weather data are forecasted.

In [ ]:
# Sameness check for other two datetime features
for column in ("forecast_datetime", "hours_ahead"):
    print(same_groups(forecast_geo_gb, column))

forecast_datetime
True     111
False      1
Name: count, dtype: int64
hours_ahead
True     111
False      1
Name: count, dtype: int64


As with `origin_datetime`, for each geographical group, the sequence of `forecast_datetime` or `hours_ahead` values is identical across the locations. Therefore, it is sufficient to check the `forecast_datetime` and `hours_ahead` features using data from just one location.

In [ ]:
# Verify the equality stated in the dataset description:
# origin_datetime + hours_ahead = forecast_datetime
unusual_sum_df = fw_subset_df[
    fw_subset_df["origin_datetime"]
    + fw_subset_df["hours_ahead"]
    != fw_subset_df["forecast_datetime"]
]
unusual_sum_df["origin_datetime"].unique()

<DatetimeArray>
['2021-10-30 02:00:00', '2021-10-31 02:00:00', '2022-03-26 01:00:00',
 '2022-03-27 01:00:00', '2022-10-29 02:00:00', '2022-10-30 02:00:00',
 '2023-03-25 01:00:00', '2023-03-26 01:00:00']
Length: 8, dtype: datetime64[ns]

Values that do not satisfy the condition occur only within the 48 hours preceding a switch to or from DST.

In [ ]:
# Check that entries not satisfying the condition will satisfy it after
# adding or subtracting respectively.
# For autumn,
# origin_datetime + hours_ahead == forecast_datetime + 1 hour.
# For spring,
# origin_datetime + hours_ahead == forecast_datetime - 1 hour.

(unusual_sum_df["origin_datetime"]
 + unusual_sum_df["hours_ahead"]
 != unusual_sum_df["forecast_datetime"]
 + pd.to_timedelta(np.where(
     unusual_sum_df["origin_datetime"].dt.month == 10,  # October
     1,  # Add 1 hour for autumn switch to DST (October)
     -1  # Subtract 1 hour for spring switch from DST (March)
 ), 'h')
).sum()

0

All conditions are satisfied after corresponding addition or subtraction.

#### data_block_id

## historical_weather_df

In [ ]:
df_show_info(historical_weather_df)

- The `data_block_id` column type is `float64`, unlike in other DataFrames.

In [ ]:
# Checking if all values in data_block_id are 
# integers to avoid errors and allow conversion to int type

print(np.unique(historical_weather_df.data_block_id.unique() %1))

- No anomalous values; all have a remainder of 0 and can be converted to `uint16` type (which can store values from 0 to 65535), as the minimum value is 1 and the maximum is 637.

- There are no missing values or duplicates in the DataFrame.
- The `data_block_id` is `float64` because the data in the original file is in "n.0" format.
- The `data_block_id` values start from 1.

## station_county_mapping

## Conclusion

- `Datetime` can be unpacked into multiple time-based features.
- `[latitude/longitude]` features represent the coordinates of the weather forecast and the weather station, which may require comparison for alignment or validation.
- Features that appear in multiple DataFrames can have different values, dtypes, and meanings (for example, `cloudcover_[low/mid/high/total]` in `forecast_weather_df` and `historical_weather_df`). It is worth renaming such features to make their differences more explicit.
- At first glance, the features `data_block_id`, `datetime`, and `row_id` appear to be correlated. Similarly, `prediction_unit_id` appears to correlate with `county`, `is_business`, and `product_type`.

## Decoding and Data Type Conversion
### Mapping Categorical Features

Nominal categorical features exist only in the train_df, client_df DataFrames.

In [ ]:
# Renaming categorical columns to improve readability
values_mapper = {
    "county": county_id_to_name_map,
    "is_business": {0: "not_business", 1: "business"},
    "is_consumption": {0: "production", 1: "consumption"},
    "product_type": {
        0: "combined",
        1: "fixed",
        2: "general_service",
        3: "spot",
    },
}

decoding_categorical_list = [train_df, client_df]

# Iterate over DataFrames with an index for tracking
for df_index, df in enumerate(decoding_categorical_list):
    for column, mapping in values_mapper.items():
        if column in df.columns:
            # Check for unexpected values
            unexpected_values = set(df[column].unique()) - set(mapping.keys())
            if unexpected_values:
                print(
                    f"\
Unexpected values in DataFrame {df_index}, column '{column}':\
{unexpected_values}"
                )
            else:
                # Apply mapping and convert to categorical type
                df[column] = df[column].map(mapping).astype("category")
    display(df.dtypes)
    display(df.head(3))

### Datetime Conversion and Reducing Memory Usage

In [ ]:
# DataFrame with the dtypes of all features as strings for simplicity
features_dtypes_df = (
    pd.concat([pd.DataFrame(df.dtypes).T for df in DF_LIST])
    .astype("str")
    .set_index(pd.Index(NAMES_LIST))
)

# Show all features
pd.set_option("display.max_columns", None)
display(features_dtypes_df)
pd.reset_option("display.max_columns")

- Only datetime-like features have object `dtype`.

In [ ]:
features_dtypes_df[features_dtypes_df == 'object'].dropna(axis=1, how='all')

In [ ]:
# renaming datetime-like features
train_df = train_df.rename(columns={"datetime": "target_datetime"})
gas_prices_df = gas_prices_df.rename(
    columns={
        "forecast_date": "gas_forecast_date",
        "origin_date": "gas_origin_date",
    }
)
client_df = client_df.rename(columns={"date": "client_date"})
electricity_prices_df = electricity_prices_df.rename(
    columns={
        "forecast_date": "electricity_forecast_datetime",
        "origin_date": "electricity_origin_date",
    }
)
forecast_weather_df = forecast_weather_df.rename(
    columns={
        "origin_datetime": "weather_origin_datetime",
        "hours_ahead": "weather_hours_ahead",
        "forecast_datetime": "weather_forecast_datetime",
    }
)
historical_weather_df = historical_weather_df.rename(columns={"datetime": "historical_datetime"})
datetime - This represents the start of the 1-hour period for which weather data are measured.

In [ ]:
forecast_weather_df[['origin_datetime', 'hours_ahead', 'forecast_datetime']].head()

In [ ]:
forecast_weather_df.groupby('origin_datetime')['hours_ahead'].unique()

In [ ]:
# forecast_weather_df[['origin_datetime', 'hours_ahead', 'forecast_datetime']]

forecast_weather_df
- origin_datetime - The timestamp of when the forecast was generated.
- hours_ahead - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
- forecast_datetime - The timestamp of the predicted weather. Generated from origin_datetime plus hours_ahead. This represents the start of the 1-hour period for which weather data are forecasted.

In [ ]:
pd.set_option("display.max_columns", None)
display(features_dtypes_df[features_dtypes_df == 'object'].dropna(axis=1, how='all'))
pd.reset_option("display.max_columns")

In [ ]:
# for df in DF_LIST:
#     df['data_block_id'] = df['data_block_id'].astype('uint16')

In [ ]:
# np.unique(features_dtypes_df.drop(columns=['county', 'is_business', 'product_type', 'is_consumption']).values.ravel())

In [ ]:
# dict.fromkeys(train_df.dtypes[train_df.dtypes == 'category'].index, 'category')

In [ ]:
# for df in DF_LIST:
#     local_mapper = {}
#     for dtype in ['float64', 'int64', 'object']:
#         # if dtype in df.dtypes.astype('str').values:
#         if dtype in df.dtypes.values:
#             local_mapper.update(dict.fromkeys(df.dtypes[df.dtypes == dtype].index, dtype))

            
#             # print(dtype)
#             # print(df.dtypes[df.dtypes == dtype])
#             # print(50 * '-')
#         else:
#             print(f'no {dtype}')
#             print(50 * '-')
#     print(local_mapper)

#     # df = df.astype(local_mapper)

In [ ]:
# pd.set_option("display.max_columns", None)
# display(features_dtypes_df.drop(columns=['county', 'is_business', 'product_type', 'is_consumption']))
# pd.reset_option("display.max_columns")

In [ ]:
# features_dtypes_df[features_dtypes_df.isin(['object'])].dropna(axis=1, how='all').columns

In [ ]:
# non_nominal_dtypes_mapper = {
#     "target": "float32",
#     # "datetime": "datetime64[ns]",
#     "data_block_id": "uint16"
#     "row_id": "uint32",
#     "prediction_unit_id": "uint8",
#     "forecast_date": "datetime64[ns]",
#     # "lowest_price_per_mwh" ':'

#     'datetime': "datetime64[ns]",
#     'forecast_date': "datetime64[ns]",
#     'origin_date': "datetime64[ns]",
#     'date': "datetime64[ns]",
#     'origin_datetime': "datetime64[ns]",
#     'forecast_datetime': "datetime64[ns]",
    
# }

train_df = train_df.astype(
    {
        "target": "float32",
        "data_block_id": "uint16",
        "row_id": "uint32",
        "prediction_unit_id": "uint8",
        "datetime": "datetime64[ns]",
    }
)

#### train_df

In [ ]:
# # Renaming to avoid confusion and improve readability
# values_mapper = {
#     "county": county_id_to_name_map,
#     "is_business": {0: "not_business", 1: "business"},
#     "is_consumption": {0: "production", 1: "consumption"},
#     "product_type": {
#         0: "combined",
#         1: "fixed",
#         2: "general_service",
#         3: "spot",
#     },
# }

# # Check for unexpected values in columns to ensure mappings are up-to-date
# for column, mapping in values_mapper.items():
#     unexpected_values = set(train_df[column].unique()) - set(mapping.keys())
#     if unexpected_values:
#         print(f"Unexpected values in {column}: {unexpected_values}")

# for key, value in values_mapper.items():
#     train_df[key] = train_df[key].map(value).astype("category")

# # Change data types to reduce memory usage
# train_df = train_df.astype(
#     {
#         "target": "float32",
#         "data_block_id": "uint16",
#         "row_id": "uint32",
#         "prediction_unit_id": "uint8",
#         "datetime": "datetime64[ns]",
#     }
# )

#### gas_prices_df

In [ ]:
# # Change data types to reduce memory usage
# gas_prices_df = gas_prices_df.astype(
#     {
#         "forecast_date": "datetime64[ns]",
#         "lowest_price_per_mwh": "float32",
#         "highest_price_per_mwh": "float32",
#         "origin_date": "datetime64[ns]",
#         "data_block_id": "uint16",
#     }
# )

# 4. EDA

### General

In [ ]:
df = train_df.copy()
df.head()

In [ ]:
df.info(show_counts=True)

- The target has missing values.

In [ ]:
df.describe().apply(lambda x: x.apply("{0:.2f}".format))

In [ ]:
# df.describe(include="object")

In [ ]:
# Adding new time-related features based on datetime

# df["hour"] = df["datetime"].dt.hour.astype("uint8")
# df["day_of_week"] = df["datetime"].dt.day_of_week.astype("uint8")
# df["day"] = df["datetime"].dt.day.astype("uint16")
# df["week_of_year"] = df["datetime"].dt.isocalendar().week.astype("int8")
# df["month"] = df["datetime"].dt.month.astype("int8")
df["month"] = df["datetime"].dt.month.astype("category")
# df["quarter"] = df["datetime"].dt.quarter.astype("int8")
# df["year"] = df["datetime"].dt.year.astype("uint16")
df["year"] = df["datetime"].dt.year.astype("category")

# df["date"] = df["datetime"].dt.date
df["date"] = df["datetime"].dt.date.astype("category")

### Target distribution

In [ ]:
# bins = 30
# plt.figure(figsize=(12, 4))


# sns.histplot(
#     data=df,
#     x='target',
#     hue='is_consumption',
#     bins=bins,
#     multiple="dodge",
#     linewidth=0.5
# )
# plt.title('Distribution of Energy Consumption (Target)')
# plt.xlabel('Energy values')
# # plt.ylabel('Frequency')

# plt.tight_layout()
# plt.show()

In [ ]:
# Creating target's discrete intervals
bins = 10
plt.figure(figsize=(12, 6))

target_bins_percentage = round(
    pd.cut(np.array(df.target), bins, precision=0).value_counts()
    / df.shape[0]
    * 100,
    2,
)

# Re-calculate first value so
# that the sum of the percentages is equal to 100.0
target_bins_percentage[0] = 100 - target_bins_percentage[1:].sum()
target_bins_percentage = [f"{i:.2f}%" for i in target_bins_percentage]

target_max = df.target.max()
ticks = range(0, int(target_max) + 1, int(target_max / bins))

ax = sns.histplot(
    df.target,
    bins=bins,
    kde=True,
    linewidth=0.5,
)

# Adding group percentage to the top of each bar
ax.bar_label(ax.containers[0], target_bins_percentage, padding=6, fontsize=11)

plt.title(
    f"Histogram of {bins} discrete bins of the target values with KDE-line"
)
plt.xticks(ticks=ticks, rotation=0)
plt.xlabel("Target values")

# Using a logarithmic scale for the y-axis for better visualization
# of small quantities of target values
plt.yscale("log")
plt.ylabel("Count, log scale", rotation=0, labelpad=45)
plt.show()

- The target distribution is non-normal, it is right-skewed.
- There are more values in the first discrete bin than the total number in the rest; the KDE-line shows that most of the values are near zero.

In [ ]:
levels = 3  # The number of boxenplot levels

# list with data for additional lines and text
# started from .25 because lower corresponding percentiles are the same

levels_list = [0.25] + np.cumsum(
    [0.5 / pow(2, i) for i in range(levels + 1)]
).tolist()
levels_values = df.target.describe(levels_list)[3:]

In [ ]:
plt.figure(figsize=(3, 8))
ax = sns.boxenplot(
    df,
    y="target",
    linewidth=0,
    k_depth=levels,
    flier_kws={
        "marker": ".",
        "s": 0.1,
    },
)

ax.set_xlim(ax.get_xlim()[0], ax.get_xlim()[1])

plt.hlines(
    levels_values.values,
    0,
    ax.get_xlim()[1],
    "orange",
    lw=1.2,
)

for ix, l in enumerate(levels_values):
    plt.text(
        ax.get_xlim()[1] + 0.03,
        levels_values.values[ix],
        f"{levels_values.index[ix]}: {levels_values.values[ix]:.2f}",
        fontsize=11,
        ha="left",
        va="center",
    )

ax.set_ylim(-0.1, 30_000)
plt.yscale("symlog", linthresh=1)
plt.title(f"Boxenplot of the target values with {levels} levels")
plt.ylabel("Target values, log scale", rotation=0, labelpad=65)
plt.yticks(fontsize=11)
plt.show()

- Q<sub>1</sub> ≈ 0.38
- Q<sub>2</sub> ≈ 31.13
- Q<sub>3</sub> ≈ 180.21
- There are only two levels on the Q<sub>1</sub> side of the boxenplot (as opposed to three levels on the Q<sub>3</sub> side), which means that there is a huge number of identical values that cannot be separated. That is, two different percentiles (6.25% and 12.5%) have the same value, which equal to the minimum value - 0.

In [ ]:
df.target.value_counts().head()

- Zero values are the most common.

In [ ]:
df.target.count()

- Zero values occur in more than 10% of the cases (351496 / 2017824).

### Add classes comparasing

### County, is_business, is_consumption and product_type combinations

From the description:

- target - The consumption or production amount for the relevant segment for the hour. The segments are defined by the county, is_business, and product_type.
- prediction_unit_id - A unique identifier for the county, is_business, and product_type combination. New prediction units can appear or disappear in the test set.

Therefore, each combination of the county, is_business, is_consumption, and product_type should be considered as a separate time series. New time series may appear or existing ones may disappear in the test set.

In [ ]:
# Count the number of target values in each time series and map each county to
# its corresponding identifier from county_id_to_name_map

df_categories = (
    df.groupby(
        [
            "county",
            "is_business",
            "product_type",
            "is_consumption",
        ],
        observed=True,
    )["target"]
    .count()
    .reset_index()
)
df_categories["county_num"] = df_categories["county"].map(
    pd.Series(county_id_to_name_map.index, county_id_to_name_map.values)
)
df_categories.head()

In [ ]:
# Create dimensions for each column

county_dim = go.parcats.Dimension(values=df_categories.county, label="County")
is_business_dim = go.parcats.Dimension(
    values=df_categories.is_business, label="Is business?"
)
product_type_dim = go.parcats.Dimension(
    values=df_categories.product_type, label="Product type"
)
is_consumption_dim = go.parcats.Dimension(
    values=df_categories.is_consumption, label="Is consumption?"
)

color = df_categories.county_num
colorscale = px.colors.make_colorscale(COLORS_LIST)

fig = go.Figure(
    data=[
        go.Parcats(
            dimensions=[
                county_dim,
                product_type_dim,
                is_business_dim,
                is_consumption_dim,
            ],
            line={"color": color, "colorscale": colorscale},
            hoveron="dimension",
            labelfont={"size": 16, "family": "sans-serif"},
            tickfont={"size": 16, "family": "sans-serif", "color": "blue"},
            arrangement="freeform",
        )
    ]
)
fig.update_layout(
    width=1200,
    height=800,
    font={"size": 18, "family": "sans-serif"},
    title="Parallel categories diagram for all observed combinations of catego\
rical features",
)
fig.show()

- There are different combinations of county-product_type-is_business features. All these combinations have two variants for is_consumption feature. What this means is that it is possible for each timestamp there are two target values for county-product_type-is_business combinations corresponding to consumption and production.
- Only two counties have one combiation of product_type-is_business features: Läänemaa and Unknown. Both of them have only 'spot' and 'business' values in corresponding features.
- It is worth noting that the description of prediction_unit_id says that new combinations of county, is_business, and product_type features may appear or disappear.

In [ ]:
# Checking the first point

# There are no timestamps in the dataframe,
# for which there are only consumption or production values.

In [ ]:
def create_prodcons_df(
    df: pd.DataFrame,
    compare: str,
    # group: list[str],
) -> pd.DataFrame:
    df = (
        df.loc[df["is_consumption"] == compare]
        .groupby(
            ["datetime", "county", "is_business", "product_type"],
            observed=True,
        )["target"]
        .count()
        .reset_index()
    )
    return df

In [ ]:
create_prodcons_df(df, "consumption")

In [ ]:
create_prodcons_df(df, "consumption").equals(
    create_prodcons_df(df, "production")
)

Yes, the first point is correct.
For each combinations of datetime-county-is_business-product_type features there are one consumption value and one production value (including nan values).

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
sns.scatterplot(
    data=df,
    x="datetime",
    y="target",
    s=1,
)

plt.title("Target values timeline")
plt.xlabel("Date")
plt.ylabel("Target values", rotation=0, labelpad=40)
plt.grid(alpha=0.3)
plt.show()

- Values less than 1500 are indistinguishable, the point density is too high for this plot.
- The target variable has seasonal and weekly cycles.
- There are 'voids' during the New Year holidays.

In [ ]:
# Consumption values are multiplied by (-1) for better visualisation

df["modified_target"] = np.where(
    df["is_consumption"] == "consumption",
    df["target"].mul(-1),
    df["target"],
)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
sns.scatterplot(
    df.groupby(
        ["date", "is_consumption", "county", "product_type"], observed=True
    )["modified_target"]
    .mean()
    .reset_index(name="modified_target"),
    # df,
    x="date",
    # x="datetime",
    y="modified_target",
    hue="county",
    palette=PALETTE,
    s=10,
)

plt.title(
    "Average energy consumption (below 0) or production (above 0) for each \
day for each county",
    fontsize=13,
)

plt.legend(
    title="County",
    title_fontsize=12,
    bbox_to_anchor=(1.005, 1),
    loc="upper left",
    borderaxespad=0,
    markerscale=3,
    frameon=False,
    fontsize=11,
)

months_locator = mdates.MonthLocator()
ax.xaxis.set_major_locator(months_locator)

plt.xticks(rotation=45)
plt.xlabel("Date")
plt.ylabel("Modified target", rotation=0, labelpad=50)
plt.grid(alpha=0.3)
plt.show()

1. Electricity consumption and production values are increasing from year to year.
2. Energy production in winter is significantly lower than in summer.
3. Energy consumption in winter is bigger than in summer.
4. During the New Year holidays, there is a decrease in electricity consumption.
5. Harjumaa county has highest average (per day) values of energy consumption (for the entire observation period) and production (for spring-autumn period). Tartumaa has second highest values for the entire observation period except last two weeks when Valgamaa get ahead in energy production.

In [ ]:
# Checking the first point

# Electricity consumption and production values are
# increasing from year to year.

In [ ]:
# Due to only one year being fully available,
# the other two years will be compared with respective months of 2022.

In [ ]:
# def create_target_comparison_by_year(
#     df: pd.DataFrame,
#     first_year: int,
#     second_year: int,
# ) -> pd.DataFrame:

#     if df[df["year"] == first_year].month.unique().size == 12:
#         complete_year = first_year
#         incomplete_year = second_year
#     else:
#         complete_year = second_year
#         incomplete_year = first_year

#     grouped = df.loc[
#         df["year"].isin([incomplete_year, complete_year])
#         & df.month.isin(df[df["year"] == incomplete_year].month.unique())
#     ].groupby(
#         [
#             "is_consumption",
#             "year",
#         ],
#         observed=True,
#     )[
#         "target"
#     ]

#     return (
#         pd.merge(
#             grouped.describe(), grouped.sum(), on=["is_consumption", "year"]
#         )
#         .rename(columns={"target": "sum"})
#         .reset_index()
#         .melt(id_vars=["year", "is_consumption"])
#     )

In [ ]:
# for year in [2021, 2023]:
#     ax = sns.catplot(
#         create_target_comparison_by_year(df, year, 2022),
#         kind="bar",
#         x="year",
#         y="value",
#         col="variable",
#         hue="is_consumption",
#         sharey=False,
#         height=2.7,
#         aspect=0.6,
#     ).set_titles("{col_name}")
#     ax.fig.subplots_adjust(top=0.8)
#     if year > 2022:
#         ax.fig.suptitle(
#             f"Comparing descriptive statistics between 2022 and \
# {year}"
#         )
#     else:
#         ax.fig.suptitle(
#             f"Comparing descriptive statistics between {year} and \
# 2022"
#         )
#     plt.show()

Yes, the first point is correct.
- Total sum, quartiles, means and maximum values increase from year to year. The standard deviation also increase. The minimum value does not change; it is zero. None of the descriptive statistics decrease.

In [ ]:
# Checking the second point
# It appears that the relative growth rate of energy production is
# growing faster than the relative growth rate of consumption.

In [ ]:
# def calculate_difference_percentage(
#     base_df: pd.DataFrame,
#     first_year: int,
#     second_year: int,
# ) -> pd.DataFrame:

#     df = create_target_comparison_by_year(base_df, first_year, second_year)
#     # .sort_values(by=["is_consumption", "year" , "variable"])
#     # df = df[df["variable"].isin(["50%", "max", "mean", "sum"])]

#     first_df = df[df["year"] == first_year].reset_index(drop=True)
#     second_df = df[df["year"] == second_year].reset_index(drop=True)

#     percent = (
#         (second_df.value - first_df.value) / first_df.value * 100
#     ).rename("percentage_difference")
#     second_df = pd.concat(
#         [
#             second_df.drop(columns=["value"]),
#             first_df["value"].rename("previous_value"),
#             second_df.value,
#             percent,
#         ],
#         axis=1,
#     )

#     return second_df

    
# calculate_difference_percentage(df, 2022, 2023)

In [ ]:
# percentage_df = pd.concat([
#     calculate_difference_percentage(df, 2022, 2023),
#     calculate_difference_percentage(df, 2021, 2022),
# ]).reset_index(drop=True)

# ax = sns.catplot(
#     percentage_df,
#     x="year",
#     y="percentage_difference",
#     row="is_consumption",
#     col="variable",
#     kind="bar",
#     hue='is_consumption',
#     sharey=False,
#     height=2.5,
#     aspect=0.6,
#     # height=3,
#     # aspect=1,
#     margin_titles=True
# )
# ax.fig.subplots_adjust(top=0.88)
# ax.fig.suptitle('Visualisation the difference between two years in percent')
# # ax.tick_params(axis='x', rotation=30)
# plt.show()
# # The second bar of the first barplot is missing because median value for the corresponding previous year months (2021 year, months from 9 to 12) was 0 and for this case growth is uncountable in percent.
# # Yes, the third point is correct.

In [ ]:
# date_range = pd.date_range(df.datetime.min(), df.datetime.max(), 4)

# ax = sns.relplot(
#     data=df,
#     x="datetime",
#     y="modified_target",
#     row="is_business",
#     col="product_type",
#     hue="county",
#     palette=PALETTE,
#     height=3,
#     s=3,
# )
# ax.set_titles("{col_name}")
# ax.set(xticks=date_range)
# ax.set_xticklabels(
#     date_range,
#     rotation=30,
# )
# ax.axes[0, 0].xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
# plt.show()

# ?Conclusions from relplot

In [ ]:
plt.figure(figsize=(10, 8))
plt.rcParams["patch.edgecolor"] = "none"

ax = sns.histplot(
    data=df[
        [
            "county",
            "product_type",
        ]
    ],
    y="county",
    hue="product_type",
    multiple="stack",
    shrink=0.75,
    palette="deep",
)

sns.move_legend(
    ax,
    "upper left",
    bbox_to_anchor=(1, 1),
    title="product_type",
    frameon=False,
)

plt.title(
    "Counties by product type",
)
plt.xticks(
    ticks=range(0, int(2.5e5), int(2.5e4)),
)
plt.xlabel(
    "Number of target values",
    fontsize=11,
)
plt.ylabel("Estonian counties", rotation=0, labelpad=60, fontsize=11)

plt.show()
plt.rcParams["patch.edgecolor"] = "black"

1. Counties have different numbers of total records and ratios of contract types.
2. First place by the count of records is the 'Spot' product type, the second place is 'Fixed' product type. Third place is most likely the 'Combined' product type.
3. It seems that for the "spot" product type, the number of records for all counties is approximately 60000, except for Läänemaa and Unknown counties, where the number of records is approximately 30000.
4. Some counties don't have certain types of contracts. In the records of Läänemaa and Unknown county there is only one type of product.

In [ ]:
# Checking the second point
# First place by the count of records is the 'Spot' product type,
# the second place is 'Fixed' product type.
# Third place is most likely the 'Combined' product type.

In [ ]:
df.groupby(["product_type"], observed=True)[["target"]].count().sort_values(
    "target", ascending=False
)

Yes, the second point is correct.

In [ ]:
# Checking the third point
# It seems that for the "spot" product type, the number of records
# for all counties is approximately 60000, except for Läänemaa and
# Unknown counties, where the number of records is approximately 30000.

In [ ]:
df.groupby(["county", "product_type"], observed=True)[
    ["target"]
].count().query('product_type == "Spot"').target.unique()

Yes, the third point is correct.

Comparing the total number of records in each subgroup (all combinations of "county", "product_type", "is_business", "is_consumption" that appear in the dataframe) due to the fact that counties have different total number of records and ratios of contract types.

In [ ]:
# Reordering columns
# df_categories = df_categories.copy()[[
#     "county",
#     "product_type",
#     "is_business",
#     "is_consumption",
#     "target",
#     "county_num",
# ]]

In [ ]:
df_categories["subgroup"] = (
    df_categories[["county", "product_type", "is_business", "is_consumption"]]
    .astype(str)
    .agg("-".join, axis=1)
)
df_categories.head()

In [ ]:
# count_df = (
#     df.groupby([
#         "county",
#         "product_type",
#         "is_business",
#         "is_consumption",
#     ], observed=True,)["target"].count().reset_index()
# )
# count_df["subgroup"] = (
#     count_df[["county", "product_type", "is_business", "is_consumption"]]
#     .astype(str)
#     .agg("-".join, axis=1)
# )
# count_df.head()

# first_part_df = count_df[count_df['county'].isin(count_df.county.unique()[:int(count_df.county.nunique() / 2)])]
# second_part_df = count_df[count_df['county'].isin(count_df.county.unique()[int(count_df.county.nunique() / 2):])]

In [ ]:
df_categories

In [ ]:
first_part_df = df_categories[
    df_categories["county"].isin(
        df_categories.county.unique()[
            : int(df_categories.county.nunique() / 2)
        ]
    )
]
second_part_df = df_categories[
    df_categories["county"].isin(
        df_categories.county.unique()[
            int(df_categories.county.nunique() / 2) :
        ]
    )
]

In [ ]:
rows = 1
cols = 2
categories = [first_part_df, second_part_df]

fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 14))
plt.subplots_adjust(
    wspace=1.1,
)
for ind, column in enumerate(categories):
    plt.subplot(rows, cols, ind + 1)
    data = categories[ind]
    ax = sns.barplot(
        data=data,
        x="target",
        y="subgroup",
        hue="county",
        palette=PALETTE,
    )
    ax.set_yticks([i for i in range(data.subgroup.nunique())])
    ax.set_yticklabels(
        data.subgroup.apply(lambda x: "-".join(x.split("-")[-3:])),
    )
    if ind == 0:
        ax.legend_.remove()
    else:
        sns.move_legend(
            ax,
            "upper left",
            bbox_to_anchor=(1, 1),
            title="County",
            frameon=False,
        )
    plt.ylabel(
        ylabel="",
    )
plt.show()

In [ ]:
df_categories["target"].value_counts()

Each combination of county, product_type, and is_business has the same number of target values for both consumption and production.

In [ ]:
# rows = 4
# cols = 1
# categories = ["county", "is_business", "product_type", "is_consumption"]

# fig, ax = plt.subplots(
#     nrows=rows,
#     ncols=cols,
#     figsize=(12, 15),
# )

# for ind, column in enumerate(categories):
#     plt.subplot(rows, cols, ind + 1)
#     data = (
#         df[column]
#         .value_counts(normalize=True)
#         .rename("percentage")
#         .mul(100)
#         .reset_index()
#         .round(2)
#     )

#     # Rounding for total sum == 100.0
#     data["percentage"] = data["percentage"].transform(
#         lambda x: pd.Series(
#             {x.index[0]: (100 - x.iloc[1:].sum())}
#         ).combine_first(x)
#     )

#     barplot = sns.barplot(
#         data=data,
#         y=column,
#         x="percentage",
#         hue=column,
#         orient="h",
#         legend=False,
#         # palette=PALETTE,
#     )
#     for container in barplot.containers:
#         barplot.bar_label(
#             container,
#             fmt=f"%.{2}f",
#         )

#     # plt.legend('', frameon=False)
#     plt.ylabel(
#         column,
#         rotation=0,
#         labelpad=60,
#     )

# fig.align_ylabels()
# plt.show()

# ?Conclusions from barplot

In [ ]:
# target_business = sns.relplot(
#     data=df,
#     x="datetime",
#     y="modified_target",
#     col="is_business",
#     hue="county",
#     height = 8,
#     # aspect = 1.6,
#     # size="size",
#     # style="sex",
#     palette=PALETTE,
#     s=1,
# )
# # l = target_business._legend

In [ ]:
# fig, ax = plt.subplots(figsize=(18, 8))
# sns.boxenplot(
#     data=df,
#     x="county",
#     y="target",
#     hue="is_consumption",
#     # k_depth = 'full'
#     # split=True,
#     # style='is_business',
# )

# plt.title(
#     "Comparison of energy production and consumption in each county"
# )
# # plt.legend(
# #     bbox_to_anchor=(1.005, 1),
# #     loc="upper left",
# #     borderaxespad=0,
# #     # markerscale=3,
# # )

# for i in range(df.county.nunique()):
#     ax.axvline(
#         i -.5,
#         color="black",
#         alpha=.2
#     )

# plt.xticks(rotation=45)

# plt.yscale('log')
# # plt.grid(axis='y',alpha=.2)

# plt.show()

In [ ]:
data = (
    df.groupby(["date", "product_type", "county"], observed=True)["target"]
    .mean()
    .reset_index(name="target")
)
data = data.astype({"date": "datetime64[ns]"})

In [ ]:
# Visualization of missing data and zeros using lineplot of the daily average

plt.figure(figsize=(14, 8))

ax = sns.lineplot(
    data=data,
    x="date",
    y="target",
    style="product_type",
    hue="county",
    lw=1,
    palette=PALETTE,
)
ax.grid()
ax.set_yscale("log")
ax.legend(loc="upper left", bbox_to_anchor=(1.005, 1), frameon=False)
plt.show()

In [ ]:
# data.query('county == "Saaremaa" and "2021-02-01" <= date <= "2021-02-01" ')
# data.query(
#     'product_type == "general_service" and county == "Pärnumaa" and ("2021-02-01" < date < "2021-03-01")'
# )
# data.county.unique().tolist()
# df_for_missing = train_df.drop(columns=['data_block_id', 'row_id', 'prediction_unit_id']).copy()

In [ ]:
# # Renaming to avoid confusion and improve readability
# values_mapper = {
#     "county": county_id_to_name_map,
#     "is_business": {
#         0: "not_business",
#         1: "business"
#     },
#     "is_consumption": {
#         0: "production",
#         1: "consumption"
#     },
#     "product_type": {
#         0: "Combined",
#         1: "Fixed",
#         2: "General service",
#         3: "Spot",
#     },
# }

# for column in values_mapper:
#     df_for_missing[column] = df_for_missing[column].map(values_mapper[column])

# df_for_missing = df_for_missing.astype({
#     "county": "category",
#     "is_business": "category",
#     "product_type": "category",
#     "is_consumption": "category",
#     "datetime": "datetime64[ns]",
# })

In [ ]:
df_for_missing = df[
    [
        "county",
        "is_business",
        "product_type",
        "target",
        "is_consumption",
        "datetime",
    ]
].copy()

df_for_missing["hour_stamp"] = (
    (df_for_missing["datetime"] - df_for_missing["datetime"].min())
    / pd.Timedelta(hours=1)
).astype(int)

df_for_missing[
    [
        "hour_stamp",
        "datetime",
    ]
].max()

In [ ]:
df_for_missing["group_index"] = df_for_missing.groupby(
    [
        "county",
        "is_business",
        "product_type",
        "is_consumption",
    ],
    observed=True,
).ngroup()

df_for_missing = df_for_missing.sort_values(["hour_stamp", "group_index"])

In [ ]:
vl = df_for_missing.values

In [ ]:
df_for_missing.head()

In [ ]:
df_for_missing

In [ ]:
missmap = np.empty(
    (
        df_for_missing.hour_stamp.max() + 1,
        df_for_missing["group_index"].nunique(),
    )
)
missmap.fill(np.nan)
for obs in df_for_missing.values:
    missmap[int(obs[6]), (obs[7])] = 0 if obs[3] == 0 else 1
missmap = missmap.T

In [ ]:
# sns.heatmap(
#     [
#         [1, 0, 0, 0, 1],
#         [1, np.NaN, np.NaN, np.NaN, 1],
#         [1, 1, 1, 1, 1],
#     ],
#     # cmap="Paired",
#     cmap='viridis',
#     cbar=False,
# )

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 80))
# sns.heatmap(
#     missmap,
#     # cmap='Paired',
#     cmap="viridis",
#     cbar=False,
# )
# missmap = missmap.T
plt.figure(figsize=(20, 8))
sns.heatmap(
    missmap,
    # cmap='Paired',
    cmap="viridis",
    cbar=True,
)
plt.show()

In [ ]:
test = df.assign(
    category_index=df.groupby(
        [
            "county",
            "is_business",
            "product_type",
            "is_consumption",
        ],
        observed=True,
    ).ngroup()
)[["datetime", "target", "category_index"]]
test["datetime"] = (
    (test["datetime"] - test["datetime"].min()) / pd.Timedelta(hours=1)
).astype(int)
test = test.rename(columns={"datetime": "hour_stamp"})
test.target = test.target.where(((test.target == 0) | (test.target.isna())), 1)
test = test.pivot(
    columns="category_index", index="hour_stamp", values="target"
)

# test.head()

In [ ]:
from matplotlib.colors import ListedColormap

plt.figure(figsize=(24, 8))

cmap = ListedColormap(["y", "forestgreen"])
xticks = 19  # Desired number -1
max_hour_range = test.index[-1]

ax = sns.heatmap(
    test.T,
    cmap=cmap,
    cbar_kws={
        "shrink": 0.5,
        "pad": 0.01,
        "aspect": 25,
        "ticks": [0.25, 0.75],
    },
)
colorbar = ax.collections[0].colorbar
colorbar.set_ticklabels(["0", ">0"])

plt.xticks(
    ticks=range(0, max_hour_range, int(max_hour_range / xticks)),
    labels=range(0, max_hour_range, int(max_hour_range / xticks)),
    rotation=45,
)
plt.show()

# ?Conclusions from heatmap

In [ ]:
def target_percentage_in_subgroup(
    df: pd.DataFrame, feature: str, broken_down_by: str, target: str
) -> pd.DataFrame:
    df = (
        df.groupby([broken_down_by, feature], observed=True)[[target]]
        .sum()
        .groupby(level=0, observed=True)
        .apply(lambda x: x * 100 / x.sum())
        .reset_index(level=0, drop=True)
        .reset_index()
        .rename(columns={"target": "percentage"})
    )
    return df

In [ ]:
def plot_for_categorical_feature(
    df: pd.DataFrame,
    broken_down_by: str,
    features_list: list[str],
    target: str,
):
    length = len(features_list)
    fig, axs = plt.subplots(
        nrows=length,
        ncols=1,
        figsize=(16, 15),
    )
    fig.tight_layout(pad=5)

    for idx, feature in enumerate(features_list):
        plt.subplot(length, 1, 1 + idx)
        data = target_percentage_in_subgroup(
            df, feature, broken_down_by, target
        )

        barplot = sns.barplot(
            data=data,
            x="percentage",
            y=feature,
            hue=broken_down_by,
        )

        for container in barplot.containers:
            barplot.bar_label(
                container,
                fmt="%.2f",
            )

        plt.title(f"{feature} broken down by {broken_down_by}")
    plt.show()

In [ ]:
plot_for_categorical_feature(
    df,
    "is_consumption",
    ["county", "is_business", "product_type"],
    "target",
)

# ?Conclusions from barplots

In [ ]:
# Feature Engineering

In [ ]:
# df_hours = df.copy()

# hours_ago = (
#     [i for i in range(1, 25)]
#     + [24 * i for i in range(2, 8)]
#     + [168 * i for i in range(2, 9)]
#     + [672 * i for i in range(3, 13)]
# )
# for h in hours_ago:
#     df_hours[f"tm_{h}h"] = df_hours["modified_target"].shift(h)
# df_hours.info()

In [ ]:
# df["tm_1h"] = df["modified_target"].shift(1)

In [ ]:
# def add_lags(df):
#     target_map = df['PJME_MW'].to_dict()
#     df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
#     df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
#     df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
#     return df
# df_label = pd.get_dummies(df_label, drop_first=True)
# df_label.info()

In [ ]:
# X = df_label.drop(
#     columns=[
#         "target",
#         "data_block_id",
#         "row_id",
#         "prediction_unit_id",
#         "modified_target",
#     ],
#     axis=1,
# )

# y = df_label["modified_target"].values

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.20, random_state=RAND
# )

# st = StandardScaler()
# X_train_std = st.fit_transform(X_train)
# X_test_std = st.transform(X_test)

In [ ]:
# def r2_adjusted(
#     y_true: np.ndarray, y_pred: np.ndarray, X_test: np.ndarray | int
# ) -> float:
#     """Коэффициент детерминации (множественная регрессия)"""
#     N_objects = len(y_true)

#     if isinstance(X_test, np.ndarray):
#         N_features = X_test.shape[1]
#     else:
#         N_features = X_test

#     #     N_features = X_test.shape[1]
#     r2 = r2_score(y_true, y_pred)
#     return 1 - (1 - r2) * (N_objects - 1) / (N_objects - N_features - 1)


# def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Mean percentage error"""
#     return np.mean((y_true - y_pred) / y_true, axis=0) * 100


# def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Mean absolute percentage error"""
#     return np.mean(np.abs((y_pred - y_true) / y_true), axis=0) * 100


# def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Weighted Absolute Percent Error"""
#     return np.sum(np.abs(y_pred - y_true)) / np.sum(y_true) * 100


# def huber_loss(
#     y_true: np.ndarray | pd.DataFrame,
#     y_pred: np.ndarray | pd.DataFrame,
#     delta: float = 1.345,
# ):
#     """Функция ошибки Хьюбера"""

#     if isinstance(y_true, pd.DataFrame):
#         y_true = y_true.squeeze().to_numpy()
#     if isinstance(y_pred, pd.DataFrame):
#         y_pred = y_pred.squeeze().to_numpy()

#     assert len(y_true) == len(y_pred), "Разные размеры данных"
#     huber_sum = 0
#     for i in range(len(y_true)):
#         if abs(y_true[i] - y_pred[i]) <= delta:
#             huber_sum += 0.5 * (y_true[i] - y_pred[i]) ** 2
#         else:
#             huber_sum += delta * (abs(y_true[i] - y_pred[i]) - 0.5 * delta)
#     huber_sum /= len(y_true)
#     return huber_sum


# def logcosh(y_true: np.ndarray, y_pred: np.ndarray):
#     """функция ошибки Лог-Кош"""
#     return np.sum(np.log(np.cosh(y_true - y_pred)))


# def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
#     """
#     Root Mean Squared Log Error (RMSLE) metric
#     Логарифмическая ошибка средней квадратичной ошибки
#     """
#     try:
#         return np.sqrt(mean_squared_log_error(y_true, y_pred))
#     except:
#         return None


# def get_metrics(
#     y_test: np.ndarray,
#     y_pred: np.ndarray,
#     X_test: np.ndarray,
#     name: str = None,
#     delta: float = 1.345,
# ):
#     """Генерация таблицы с метриками"""
#     df_metrics = pd.DataFrame()
#     df_metrics["model"] = [name]

#     df_metrics["MAE"] = mean_absolute_error(y_test, y_pred)
#     df_metrics["MSE"] = mean_squared_error(y_test, y_pred)
#     df_metrics["Huber_loss"] = huber_loss(y_test, y_pred, delta)
#     df_metrics["Logcosh"] = logcosh(y_test, y_pred)
#     df_metrics["RMSE"] = np.sqrt(mean_squared_error(y_test, y_pred))
#     df_metrics["RMSLE"] = rmsle(y_test, y_pred)
#     df_metrics["R2 adjusted"] = r2_adjusted(y_test, y_pred, X_test)
#     df_metrics["MPE_%"] = mpe(y_test, y_pred)
#     df_metrics["MAPE_%"] = mape(y_test, y_pred)
#     df_metrics["WAPE_%"] = wape(y_test, y_pred)

#     return df_metrics

In [ ]:
# lr_skl = LinearRegression()
# lr_skl.fit(X_train_std, y_train)

In [ ]:
# lr_skl_pred = lr_skl.predict(X_test_std)
# skl_m = get_metrics(y_test, lr_skl_pred, X_test_std, name="skl_lr")
# skl_m

In [ ]:
# df_hours = df_hours[
#     [
#         "county",
#         "is_business",
#         "product_type",
#         "target",
#         "is_consumption",
#         "hour",
#         "day_of_week",
#         "day",
#         "week_of_year",
#         "month",
#         "quarter",
#         "year",
#         "modified_target",
#         "tm_1h",
#         "tm_2h",
#         "tm_3h",
#         "tm_4h",
#         "tm_5h",
#         "tm_6h",
#         "tm_7h",
#         "tm_8h",
#         "tm_9h",
#         "tm_10h",
#         "tm_11h",
#         "tm_12h",
#         "tm_13h",
#         "tm_14h",
#         "tm_15h",
#         "tm_16h",
#         "tm_17h",
#         "tm_18h",
#         "tm_19h",
#         "tm_20h",
#         "tm_21h",
#         "tm_22h",
#         "tm_23h",
#         "tm_24h",
#         "tm_48h",
#         "tm_72h",
#         "tm_96h",
#         "tm_120h",
#         "tm_144h",
#         "tm_168h",
#         "tm_336h",
#         "tm_504h",
#         "tm_672h",
#         "tm_840h",
#         "tm_1008h",
#         "tm_1176h",
#         "tm_1344h",
#         "tm_2016h",
#         "tm_2688h",
#         "tm_3360h",
#         "tm_4032h",
#         "tm_4704h",
#         "tm_5376h",
#         "tm_6048h",
#         "tm_6720h",
#         "tm_7392h",
#         "tm_8064h",
#     ]
# ]

In [ ]:
# df_hours = df_hours.sort_index()
# tss = TimeSeriesSplit(n_splits=3, test_size=300_000)

# fold = 0
# preds = []
# scores = []
# for train_idx, val_idx in tss.split(df_hours):
#     train = df_hours.iloc[train_idx]
#     test = df_hours.iloc[val_idx]

#     reg = XGBRegressor(
#         n_estimators=2000,
#         early_stopping_rounds=50,
#         objective="reg:squarederror",
#         enable_categorical=True,
#         eval_metric="mae",
#         # max_depth=3,
#         learning_rate=0.01,
#         random_state=RAND,
#     )
#     FEATURES = [
#         "county",
#         "is_business",
#         "product_type",
#         "is_consumption",
#         "hour",
#         "day_of_week",
#         "day",
#         "week_of_year",
#         "month",
#         "quarter",
#         "year",
#         "tm_1h",
#         "tm_2h",
#         "tm_3h",
#         "tm_4h",
#         "tm_5h",
#         "tm_6h",
#         "tm_7h",
#         "tm_8h",
#         "tm_9h",
#         "tm_10h",
#         "tm_11h",
#         "tm_12h",
#         "tm_13h",
#         "tm_14h",
#         "tm_15h",
#         "tm_16h",
#         "tm_17h",
#         "tm_18h",
#         "tm_19h",
#         "tm_20h",
#         "tm_21h",
#         "tm_22h",
#         "tm_23h",
#         "tm_24h",
#         "tm_48h",
#         "tm_72h",
#         "tm_96h",
#         "tm_120h",
#         "tm_144h",
#         "tm_168h",
#         "tm_336h",
#         "tm_504h",
#         "tm_672h",
#         "tm_840h",
#         "tm_1008h",
#         "tm_1176h",
#         "tm_1344h",
#         "tm_2016h",
#         "tm_2688h",
#         "tm_3360h",
#         "tm_4032h",
#         "tm_4704h",
#         "tm_5376h",
#         "tm_6048h",
#         "tm_6720h",
#         "tm_7392h",
#         "tm_8064h",
#     ]
#     # TARGET = "modified_target"
#     TARGET = "target"

#     X_train = train[FEATURES]
#     y_train = train[TARGET]

#     X_test = test[FEATURES]
#     y_test = test[TARGET]

#     reg.fit(
#         X_train,
#         y_train,
#         eval_set=[(X_train, y_train), (X_test, y_test)],
#         verbose=20,
#     )

#     y_pred = reg.predict(X_test)
#     preds.append(y_pred)
#     score = np.sqrt(mean_squared_error(y_test, y_pred))
#     scores.append(score)

# hours_ago = (
#     [i for i in range(1, 25)]
#     + [24 * i for i in range(2, 8)]
#     + [168 * i for i in range(2, 9)]
#     + [672 * i for i in range(3, 13)]
# )
# for h in hours_ago:
#     df[f"t_{h}h"] = df["target"].shift(h)

In [ ]:
# fig, ax = plt.subplots(figsize=(35, 30))

# sns.heatmap(df_hours.corr(), annot=True, cmap="Blues", fmt=".1f")
# # plt.figure(figsize=(25, 25))
# plt.show()
# # numeric_only=True

In [ ]:
# df = df.sort_index().sort_values(
#     ["county", "is_business", "product_type", "is_consumption"],
#     kind="mergesort",
# )
# df.sort_index()
# tss = TimeSeriesSplit(n_splits=3, test_size=300_000)

# a = {}
# for i in range(1000):
#     x = 0
#     for j in range(20):
#         x += np.random.choice([-1, 1])
#     a[x] = a.get(x, 0) + 1


# sns.barplot(x=list(a.keys()), y=list(a.values()));

In [ ]:
# min(data.keys())
# max(data.keys())
# len(data.keys())
# {k: 0 for (k, 0) in range(min(data.keys()), max(data.keys())) if not in data.keys()}
# {k: v*2 for (k,v) in dict1.items()}
# {key:value for (key,value) in dictonary.items()}
# zip()

# for train_idx, val_idx in tss.split(df):
# print('1 train:', train_idx)
# display(df.iloc[train_idx].tail(5))
# print('1 val:', val_idx)

In [ ]:
# tss = TimeSeriesSplit(n_splits=3, test_size=300_000)
# df = df.sort_index()

# fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
# fold = 0

# for train_idx, val_idx in tss.split(df):
#     train = df.iloc[train_idx]
#     test = df.iloc[val_idx]
#     train["modified_target"].plot(
#         ax=axs[fold],
#         label="Training Set",
#         title=f"Data Train/Test Split Fold {fold}",
#     )
#     test["modified_target"].plot(ax=axs[fold], label="Test Set")
#     axs[fold].axvline(test.index.min(), color="black", ls="--")
#     fold += 1
# plt.show()

In [ ]:
# fold = 0
# preds = []
# scores = []
# for train_idx, val_idx in tss.split(df):
#     train = df.iloc[train_idx]
#     test = df.iloc[val_idx]

#     reg = XGBRegressor(
#         n_estimators=2000,
#         early_stopping_rounds=50,
#         objective="reg:squarederror",
#         enable_categorical=True,
#         eval_metric="mae",
#         # max_depth=3,
#         learning_rate=0.01,
#         random_state=RAND,
#     )
#     FEATURES = [
#         "county",
#         "is_business",
#         "product_type",
#         # 'target',
#         "is_consumption",
#         # 'data_block_id',
#         # 'row_id',
#         # 'prediction_unit_id',
#         "hour",
#         "day_of_week",
#         "day",
#         "week_of_year",
#         "month",
#         "quarter",
#         "year",
#         # 'modified_target',
#     ]
#     TARGET = "modified_target"

#     X_train = train[FEATURES]
#     y_train = train[TARGET]

#     X_test = test[FEATURES]
#     y_test = test[TARGET]

#     reg.fit(
#         X_train,
#         y_train,
#         eval_set=[(X_train, y_train), (X_test, y_test)],
#         verbose=20,
#     )

#     y_pred = reg.predict(X_test)
#     preds.append(y_pred)
#     score = np.sqrt(mean_squared_error(y_test, y_pred))
#     scores.append(score)